In [4]:
from scapy.all import *
import socket
import time
import random

# Resolve the domain to an IP
target_ip = socket.gethostbyname('itu.edu.tr')

def send_tcp_packet(dst_ip, dst_port, flags, tos=0x00, payload=None):
    sport = random.randint(1024, 65535)
    seq = random.randint(0, 1000000)
    ip = IP(dst=dst_ip, tos=tos)
    tcp = TCP(sport=sport, dport=dst_port, flags=flags, seq=seq)
    packet = ip/tcp
    if payload:
        packet = packet/Raw(load=payload)
    send(packet)
    time.sleep(2)

def send_quic_packet(dst_ip, dst_port, tos=0x00):
    sport = random.randint(1024, 65535)
    udp = UDP(sport=sport, dport=dst_port)
    payload = b'\x43\x02'  # Example QUIC-like payload; actual QUIC payloads are more complex
    packet = IP(dst=dst_ip, tos=tos)/udp/Raw(load=payload)
    send(packet)
    time.sleep(2)

# Step 1: Send a regular ICMP ping
print(f"Sending regular ping to {target_ip}")
icmp_ping = IP(dst=target_ip)/ICMP()
send(icmp_ping)
time.sleep(2)

# Step 2: Send an ICMP echo request with ECT(0)
print(f"Sending ICMP Echo Request with ECT(0) to {target_ip}")
send(IP(dst=target_ip, tos=0x02)/ICMP())
time.sleep(2)

# Step 3: Send a TCP SYN packet with ECT(0) to port 443 (HTTPS port)
print(f"Sending TCP SYN packet with ECT(0) to {target_ip} on port 443")
send_tcp_packet(target_ip, 443, 'S', 0x02)

# Step 4: Send a TCP packet with SYN + ECN-Echo (E) flag set with ECT(0)
print("Sending TCP packet with SYN + ECN-Echo flags and ECT(0) to port 443")
send_tcp_packet(target_ip, 443, 'SE', 0x02)

# Step 5: Send a TCP packet with SYN + ECN-Echo + CWR flags set with ECT(0)
print("Sending TCP packet with SYN + ECN-Echo + CWR flags and ECT(0) to port 443")
send_tcp_packet(target_ip, 443, 'SEC', 0x02)

# Step 6: Send a non-ECT TCP packet with SYN + ECN-Echo + CWR flags set to port 443
print("Sending TCP packet with SYN + ECN-Echo + CWR flags without ECT to port 443")
send_tcp_packet(target_ip, 443, 'SEC')

# Step 7: Send a non-ECT TCP packet with SYN + ECN-Echo flags set to port 443
print("Sending TCP packet with SYN + ECN-Echo flags without ECT to port 443")
send_tcp_packet(target_ip, 443, 'SE')

# Step 8: Send a QUIC packet with ECT(0)
print("Sending QUIC packet with ECT(0) to port 443")
send_quic_packet(target_ip, 443, 0x02)

print("Done.")


Sending regular ping to 194.1.147.65

Sent 1 packets.
.Sending ICMP Echo Request with ECT(0) to 194.1.147.65

Sent 1 packets.
.Sending HTTPS GET request (SYN packet only) with ECT(0) to 194.1.147.65

Sent 1 packets.
.Sending TCP packet with SYN + ECN-Echo flags and ECT(0)

Sent 1 packets.
.Sending TCP packet with SYN + ECN-Echo + CWR flags and ECT(0)

Sent 1 packets.
.Sending TCP packet with SYN + ECN-Echo + CWR flags without ECT

Sent 1 packets.
.Sending TCP packet with SYN + ECN-Echo flags without ECT

Sent 1 packets.
.Done.
